# 3 Way Soccer

In this notebook we will use the ThreeWaySoccerEnv to check various betting strategies.
For data, we will use the free data provided by www.football-data.co.uk.
Specifically for this notebook we will use the English Premier League.

## Getting the data

The site keeps the data in CSV files, which we can easily load with pandas as such:

In [1]:
import pandas as pd
raw_18_19_odds_data = pd.read_csv('http://www.football-data.co.uk/mmz4281/1819/E0.csv')

Let's take a look at the first 5 rows of the data:

In [2]:
raw_18_19_odds_data.head(5)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,10/08/2018,Man United,Leicester,2,1,H,1,0,H,...,1.79,17,-0.75,1.75,1.70,2.29,2.21,1.55,4.07,7.69
1,E0,11/08/2018,Bournemouth,Cardiff,2,0,H,1,0,H,...,1.83,20,-0.75,2.20,2.13,1.80,1.75,1.88,3.61,4.70
2,E0,11/08/2018,Fulham,Crystal Palace,0,2,A,0,1,A,...,1.87,22,-0.25,2.18,2.11,1.81,1.77,2.62,3.38,2.90
3,E0,11/08/2018,Huddersfield,Chelsea,0,3,A,0,2,A,...,1.84,23,1.00,1.84,1.80,2.13,2.06,7.24,3.95,1.58
4,E0,11/08/2018,Newcastle,Tottenham,1,2,A,1,2,A,...,1.81,20,0.25,2.20,2.12,1.80,1.76,4.74,3.53,1.89


As you can see, we have alot of data to work with. We only need the team names, the odds and the result.
For the odds, we will use the maximum betting odds from betbrain.com (columns BbMxH, BbMxA and BbMxD for maximum home odds, maximum away odds and maximum draw odds.
As for the result, we will use the the FTR (Full Time Result) column, and change the values from (H, A, D) to (0, 1, 2).

In [3]:
odds_dataframe = raw_18_19_odds_data[['HomeTeam', 'AwayTeam', 'BbMxH', 'BbMxA', 'BbMxD']]
odds_dataframe.rename({'HomeTeam' :'home_team', 'AwayTeam': 'away_team', 'BbMxH': 'home',
                       'BbMxA': 'away', 'BbMxD': 'draw'}, axis='columns', inplace=True)
odds_dataframe['result'] = raw_18_19_odds_data['FTR'].map({'H': 0, 'A': 1, 'D': 2})

/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Let's take a look at the first 5 rows of our odds dataframe:

In [4]:
odds_dataframe.head(5)

,home_team,away_team,home,away,draw,result
0,Man United,Leicester,1.60,8.05,4.20,0
1,Bournemouth,Cardiff,1.93,4.75,3.71,0
2,Fulham,Crystal Palace,2.60,3.05,3.49,1
3,Huddersfield,Chelsea,6.85,1.66,4.07,1
4,Newcastle,Tottenham,4.01,2.12,3.57,1


Now we can use our custom environment and pass to it our odds dataframe:

In [5]:
from oddsgym.envs.soccer import ThreeWaySoccerOddsEnv
env = ThreeWaySoccerOddsEnv(odds_dataframe)
max_steps_limit = odds_dataframe.shape[0]

Let's see what happens when we only bet on the home team:

In [6]:
env.reset()
for _ in range(1, max_steps_limit):
    print(env.render())
    obs, reward, done, info = env.step((0,))
    if done:
        break

Home Team Man United VS Away Team Leicester. Current balance at step 0: 10
Home Team Bournemouth VS Away Team Cardiff. Current balance at step 1: 10.6
Home Team Fulham VS Away Team Crystal Palace. Current balance at step 2: 11.53
Home Team Huddersfield VS Away Team Chelsea. Current balance at step 3: 10.53
Home Team Newcastle VS Away Team Tottenham. Current balance at step 4: 9.53
Home Team Watford VS Away Team Brighton. Current balance at step 5: 8.53
Home Team Wolves VS Away Team Everton. Current balance at step 6: 10.01
Home Team Arsenal VS Away Team Man City. Current balance at step 7: 9.01
Home Team Liverpool VS Away Team West Ham. Current balance at step 8: 8.01
Home Team Southampton VS Away Team Burnley. Current balance at step 9: 8.3
Home Team Cardiff VS Away Team Newcastle. Current balance at step 10: 7.300000000000001
Home Team Chelsea VS Away Team Arsenal. Current balance at step 11: 6.300000000000001
Home Team Everton VS Away Team Southampton. Current balance at step 12: 7.

We can see that after 60 games, we strike out. Let's try a random gambler:

In [7]:
from random import randint
env.reset()
for _ in range(1, max_steps_limit):
    print(env.render())
    obs, reward, done, info = env.step((randint(0, 2),))
    if done:
        break

Home Team Man United VS Away Team Leicester. Current balance at step 0: 10
Home Team Bournemouth VS Away Team Cardiff. Current balance at step 1: 10.6
Home Team Fulham VS Away Team Crystal Palace. Current balance at step 2: 11.53
Home Team Huddersfield VS Away Team Chelsea. Current balance at step 3: 10.53
Home Team Newcastle VS Away Team Tottenham. Current balance at step 4: 9.53
Home Team Watford VS Away Team Brighton. Current balance at step 5: 8.53
Home Team Wolves VS Away Team Everton. Current balance at step 6: 7.529999999999999
Home Team Arsenal VS Away Team Man City. Current balance at step 7: 6.529999999999999
Home Team Liverpool VS Away Team West Ham. Current balance at step 8: 5.529999999999999
Home Team Southampton VS Away Team Burnley. Current balance at step 9: 4.529999999999999
Home Team Cardiff VS Away Team Newcastle. Current balance at step 10: 7.139999999999999
Home Team Chelsea VS Away Team Arsenal. Current balance at step 11: 9.349999999999998
Home Team Everton VS A